In [26]:
import torch

inputs = torch.tensor([
    [0.43,0.15,0.89],       # your
    [0.55,0.87,0.66],       # journey
    [0.57,0.85,0.64],       # starts
    [0.22,0.58,0.33],       # with
    [0.77,0.25,0.10],       # one
    [0.05,0.80,0.55]        # step
])

In [27]:
x_2= inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [28]:
torch.manual_seed(123)
w_q = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_k = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_v = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [29]:
print(w_q,w_k,w_v)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]]) Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]]) Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [30]:
q2 =x_2 @ w_q
k2 =x_2 @ w_k
v2 =x_2 @ w_v
print(q2,v2,k2)

tensor([0.4306, 1.4551]) tensor([0.3951, 1.0037]) tensor([0.4433, 1.1419])


In [31]:
# for whole inputs
query = inputs @ w_q
key = inputs @ w_k
value = inputs @ w_v
print("query vextors are:\n",query)
print("key  vextors are:\n",key)
print("value vextors are:\n",value)

query vextors are:
 tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])
key  vextors are:
 tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
value vextors are:
 tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


In [32]:
key_2 = key[1]
attn_score_22 = q2.dot(key_2)
print(attn_score_22)

tensor(1.8524)


In [33]:
attn_score_2 = q2 @ key.T
print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [34]:
attn_score = query @ key.T
attn_score

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])

In [35]:
# scaling, division by sqrt(2) and then applying softmax
# doing for one token, index 1 token
d_k = key.shape[-1]
attn_weight_2 = torch.softmax(attn_score_2/d_k**0.5, dim=-1)
print(attn_weight_2)
print(attn_weight_2.sum())
print(d_k)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor(1.)
2


In [36]:
# calculatinf context vector for 2nd token
context_vec_2 = attn_weight_2 @ value
print(context_vec_2)

tensor([0.3061, 0.8210])


### Pyhton class for the whole process

In [37]:
import torch.nn as nn

class selfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.w_query = nn.Parameter(torch.rand(d_in,d_out))
        self.w_key = nn.Parameter(torch.rand(d_in,d_out))
        self.w_value = nn.Parameter(torch.rand(d_in,d_out))

    def forward(self, x):
        query = x @ self.w_query
        key = x @ self.w_key
        value = x @ self.w_value

        attn_score = query @ key.T
        attn_weight = torch.softmax(attn_score/key.shape[-1]**0.5, dim=-1)
        context_vec = attn_weight @ value

        return context_vec 

In [38]:
torch.manual_seed(123)
sa_c1 = selfAttention_v1(d_in=d_in,d_out=d_out)
context_vec = sa_c1(inputs)
print(context_vec)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [39]:
import torch.nn as nn

class selfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.w_query = nn.Linear(d_in,d_out , bias=qkv_bias)
        self.w_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value = nn.Linear(d_in,d_out, bias=qkv_bias)

    def forward(self, x):
        query = self.w_query(x)
        key = self.w_key(x)
        value = self.w_value(x)

        attn_score = query @ key.T
        attn_weight = torch.softmax(attn_score/key.shape[-1]**0.5, dim=-1)
        context_vec = attn_weight @ value

        return context_vec 

In [40]:
torch.manual_seed(123)
sa_v2 = selfAttention_v2(d_in=d_in,d_out=d_out)
context_vec2 = sa_v2(inputs)
print(context_vec2)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)
